In [2]:
import openpyxl
import datetime
import time

filename = "Bank Rec_2.xlsx"
wb = openpyxl.load_workbook(filename)

reconciled_bank_row_list1 = []
unreconciled_bank_row_list1 = []
final_unreconciled_bank_list = []
cummulative_reconciled_bank_rows = []


bank_sheet = wb["Bank Data"]
sage_sheet = wb["Sage Data"]
unposted_bank_sheet = wb["Unposted Bank"]
extra_in_sage_sheet = wb["Extra In Sage"]

unposted_bank_row = 2
extra_in_sage_row = 2

# Unposted Bank Headers
unposted_bank_sheet.cell(1, column=1).value = bank_sheet.cell(1, column=1).value
unposted_bank_sheet.cell(1, column=2).value = bank_sheet.cell(1, column=2).value
unposted_bank_sheet.cell(1, column=3).value = bank_sheet.cell(1, column=3).value
unposted_bank_sheet.cell(1, column=4).value = bank_sheet.cell(1, column=4).value
unposted_bank_sheet.cell(1, column=5).value = bank_sheet.cell(1, column=5).value
unposted_bank_sheet.cell(1, column=6).value = bank_sheet.cell(1, column=6).value
unposted_bank_sheet.cell(1, column=7).value = bank_sheet.cell(1, column=7).value

# Sage Headers
extra_in_sage_sheet.cell(1, column=1).value = sage_sheet.cell(1, column=1).value
extra_in_sage_sheet.cell(1, column=2).value = sage_sheet.cell(1, column=2).value
extra_in_sage_sheet.cell(1, column=3).value = sage_sheet.cell(1, column=3).value
extra_in_sage_sheet.cell(1, column=4).value = sage_sheet.cell(1, column=4).value
extra_in_sage_sheet.cell(1, column=5).value = sage_sheet.cell(1, column=5).value
extra_in_sage_sheet.cell(1, column=6).value = sage_sheet.cell(1, column=6).value
extra_in_sage_sheet.cell(1, column=7).value = sage_sheet.cell(1, column=7).value
extra_in_sage_sheet.cell(1, column=8).value = sage_sheet.cell(1, column=8).value


for bank_row in range(2, bank_sheet.max_row + 1):
    unreconciled_bank_row_list1.append(bank_row)
    # get transaction description
    bank_trans_desc_cell = bank_sheet.cell(bank_row, column=5).value
    bank_trans_split = bank_trans_desc_cell.split()
    bank_trans_join = bank_trans_split[0] + " " + bank_trans_split[1]
    bank_trans_desc_cell = bank_trans_join
    
    # get date value
    bank_date = bank_sheet.cell(bank_row, column=1).value
    date = datetime.datetime.date(bank_date)
    xdate = date.strftime("%d/%m/%Y")
    bank_date = datetime.datetime.strptime(xdate, "%d/%m/%Y").date()

    payment_type = ""

    bank_col_6 = bank_sheet.cell(bank_row, column=6).value
    bank_col_7 = bank_sheet.cell(bank_row, column=7).value
    
    ## Get Payment type
    # Paid out
    if isinstance(bank_col_6, float) or isinstance(bank_col_6, int) and bank_col_6 != None and bank_col_6 > 0:
        bank_price_cell = bank_col_6
        payment_type = "paid out"

    # Paid In
    else:
        bank_price_cell = bank_col_7
        payment_type = "paid in"
        
    ## Get Sage Value
    for sage_row in range(2, sage_sheet.max_row + 1):

        # for Pain In
        if payment_type == "paid in":
            sage_trans_desc_cell = sage_sheet.cell(sage_row, column=5).value

        # for Paid Out
        else:
            sage_trans_desc_cell = sage_sheet.cell(sage_row, column=6).value

        # Get sage date
        sage_date = sage_sheet.cell(sage_row, column=4).value
        date_object = datetime.datetime.strptime(sage_date, "%d/%m/%Y").date()
        sage_date = date_object

        # check which column contains sage price
        col_7 = sage_sheet.cell(sage_row, column=7).value
        col_8 = sage_sheet.cell(sage_row, column=8).value

        if isinstance(col_7, float) or isinstance(col_7, int) and col_7 != None and col_7 > 0:

            sage_price_cell = col_7

        elif isinstance(col_8, float) or isinstance(col_8, int) and col_8 != None and col_8 > 0:

            sage_price_cell = col_8

        ## compare values for reconciliation
        unreconciled_bank_row_list1.append(bank_row)
        #Compare the bank price with sage price, if the same reconcile
        
        if bank_row in reconciled_bank_row_list1:
            pass
        
        elif bank_price_cell == sage_price_cell and bank_date == sage_date:
            reconciled_bank_row_list1.append(bank_row)
            break
            
        if bank_row not in reconciled_bank_row_list1:

            # Compare the transaction description in both sage and bank transaction descriptions
            if bank_trans_desc_cell in sage_trans_desc_cell:


                if bank_price_cell == sage_price_cell and bank_date == sage_date:
                    reconciled_bank_row_list1.append(bank_row)

                # To check if the present bank desc price, macthes sage desc price in another cell in sage, and reconcile it

                for sage_row2 in range(2, sage_sheet.max_row + 1):

                    # for Paid In
                    if payment_type == "paid in":
                        sage_trans_desc_cell = sage_sheet.cell(sage_row2, column=5).value
                    # for Paid out
                    else:
                        sage_trans_desc_cell = sage_sheet.cell(sage_row2, column=6).value

                    # Get sage date
                    sage_date2 = sage_sheet.cell(sage_row2, column=4).value
                    date_object2 = datetime.datetime.strptime(sage_date2, "%d/%m/%Y").date()
                    sage_date2 = date_object2

                    # check which column contains sage price
                    col_7_2 = sage_sheet.cell(sage_row2, column=7).value
                    col_8_2 = sage_sheet.cell(sage_row2, column=8).value

                    if isinstance(col_7_2, float) or isinstance(col_7_2, int) and col_7_2 != None and col_7_2 > 0:

                        sage_price_cell2 = col_7_2

                    elif isinstance(col_8_2, float) or isinstance(col_8_2, int) and col_8_2 != None and col_8_2 > 0:

                        sage_price_cell2 = col_8_2


                    ## Compare values for reconcilation

                    # if the prices and dates are the same
                    if bank_price_cell == sage_price_cell2 and bank_date == sage_date2 :

                        # Get sage row, and add it to the reconciled sage row list
                        sage_row_num = sage_row2
                        reconciled_bank_row_list1.append(bank_row)
                        break

                    # if the prices are the same, but date is not the same  
                    elif bank_price_cell == sage_price_cell2 and bank_date != sage_date2:
                        unreconciled_bank_row_list1.append(bank_row)


                # Check if the bank price is greater than the sage price,
                # If it is, add up the sage price and compare it to the bank price
                if bank_price_cell > sage_price_cell:

                    total_sage_price = 0

                    # loop through the sage sheet again
                    for row3 in range(2, sage_sheet.max_row + 1):
                        
                        # for Paid In
                        if payment_type == "paid in":
                            sage_trans_desc_cell2 = sage_sheet.cell(row3, column=5).value
                    
                        # for Paid out
                        else:
                            sage_trans_desc_cell2 = sage_sheet.cell(row3, column=6).value
                        
                        
                        # compare both transaction descriptions
                        
                        if bank_trans_desc_cell in sage_trans_desc_cell2:

                            col_7_3 = sage_sheet.cell(row3, column=7).value
                            col_8_3 = sage_sheet.cell(row3, column=8).value

                            #check which column contains the sage price and add the value to the previous value
                            if isinstance(col_7_3, float) or isinstance(col_7_3, int) and col_7_3 != None and col_7_3 > 0:


                                sage_price_cell21 = col_7_3
                                total_sage_price += sage_price_cell21
                                total_sage_price = round(total_sage_price, 2)

                            elif isinstance(col_8_3, float) or isinstance(col_8_3, int) and col_8_3 != None and col_8_3 > 0:

                                sage_price_cell21 = col_8_3
                                total_sage_price += sage_price_cell21
                                total_sage_price = round(total_sage_price, 2)

                    # check if the bank price is equal to the total sage price
                    if bank_price_cell == total_sage_price:

                        reconciled_bank_row_list1.append(bank_row)

                    ## check if the bank price is less than the total sage price, and add up the total bank price of that description
                    elif bank_price_cell < total_sage_price:
                        
                        unreconciled_bank_row_list1.append(bank_row)

                        # Calculate the total bank price and compare
                        total_bank_price = 0
                        # loop to sum up all the bank prices with transaction description
                        temp_cummulative_reconciled_bank_rows = []
                        for row4 in range(2, bank_sheet.max_row + 1):

                            bank_trans_desc_cell2 = bank_sheet.cell(row4, column=5).value
                            bank_trans_split2 = bank_trans_desc_cell2.split()
                            bank_trans_join2 = bank_trans_split2[0] + " " + bank_trans_split2[1]
                            bank_trans_desc_cell2 = bank_trans_join2


                            if bank_trans_desc_cell == bank_trans_desc_cell2:

                                # Add potentially reconciled row to Temporary reconciled row list
                                temp_cummulative_reconciled_bank_rows.append(row4)

                                payment_type = ""

                                bank_col_6 = bank_sheet.cell(row4, column=6).value
                                bank_col_7 = bank_sheet.cell(row4, column=7).value

                                # Paid out
                                if isinstance(bank_col_6, float) or isinstance(bank_col_6,
                                                                               int) and bank_col_6 != None and bank_col_6 > 0:
                                    bank_price_cell2 = bank_col_6

                                    total_bank_price += bank_price_cell2
                                    total_bank_price = round(total_bank_price, 2)
                                    payment_type = "paid out"

                                # Paid In
                                else:
                                    bank_price_cell2 = bank_col_7

                                    total_bank_price += bank_price_cell2
                                    total_bank_price = round(total_bank_price, 2)
                                    payment_type = "paid in"


                        # check if the total bank price is equal to the total sage price
                        if total_bank_price == total_sage_price:
                            reconciled_bank_row_list1.append(bank_row)

                            for item in temp_cummulative_reconciled_bank_rows:
                                cummulative_reconciled_bank_rows.append(item)

                    #check is the bank price is not equal to the total_sage_price
                    elif bank_price_cell != total_sage_price:
                        unreconciled_bank_row_list1.append(bank_row)     


                elif bank_price_cell != sage_price_cell:
                    unreconciled_bank_row_list1.append(bank_row)



for item in cummulative_reconciled_bank_rows:
    reconciled_bank_row_list1.append(item)

# Check for the bank rows in the unreconciled list, that is not in the reconciled list
for item in unreconciled_bank_row_list1:
    if item not in reconciled_bank_row_list1:
        final_unreconciled_bank_list.append(item)

# Get the unique values in the list
final_unreconciled_bank_list = set(final_unreconciled_bank_list)

# fetch the unreconciled bank rows from the sage sheet and print to the extra in sage sheet
for bank_row in final_unreconciled_bank_list:
    for no in range(1):
        unposted_bank_sheet.cell(unposted_bank_row, column=1).value = bank_sheet.cell(bank_row,
                                                                                      column=1).value
        unposted_bank_sheet.cell(unposted_bank_row, column=2).value = bank_sheet.cell(bank_row,
                                                                                      column=2).value
        unposted_bank_sheet.cell(unposted_bank_row, column=3).value = bank_sheet.cell(bank_row,
                                                                                      column=3).value
        unposted_bank_sheet.cell(unposted_bank_row, column=4).value = bank_sheet.cell(bank_row,
                                                                                      column=4).value
        unposted_bank_sheet.cell(unposted_bank_row, column=5).value = bank_sheet.cell(bank_row,
                                                                                      column=5).value
        unposted_bank_sheet.cell(unposted_bank_row, column=6).value = bank_sheet.cell(bank_row,
                                                                                      column=6).value
        unposted_bank_sheet.cell(unposted_bank_row, column=7).value = bank_sheet.cell(bank_row,
                                                                                      column=7).value
    unposted_bank_row += 1
    wb.save("Finished Bank Rec.xlsx")